In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

def obtener_peajes_por_ciudades(capitales):
    # Convertir lista de capitales a array de NumPy
    capitales = np.array(capitales)
    n = len(capitales)

    # Inicializar matriz NumPy para peajes
    peajes = np.zeros((n, n))

    # Headers para la petición HTTP (importante para evitar bloqueos)
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    for i in range(n):
        for j in range(n):
            if i != j:  # No procesar cuando coincidad el origen y destino
                origen = capitales[i].replace(" ", "+")
                destino = capitales[j].replace(" ", "+")

                url = f"https://mx.mejoresrutas.com/distancias/?from={origen}&to={destino}"
                try:
                    response = requests.get(url, headers=headers)
                    soup = BeautifulSoup(response.text, "html.parser")
                    celda = soup.find(class_="l-summary-block").find_all("td")

                    # Extraer peaje
                    try:
                        peaje = celda[11].text.strip().split(" ")[0]
                        peajes[i, j] = int(peaje.replace(".", ""))

                        print(f"{origen} a {destino}: ${peaje}")
                    except:
                        peajes[i, j] = 0

                except Exception as e:
                    print(f"Error procesando {origen} a {destino}: {str(e)}")
                    continue

    return peajes

def guardar_peajes(capitales, peajes):
    # Crear DataFrame para la matriz de peajes
    df_peajes = pd.DataFrame(peajes, index=capitales, columns=capitales)

    # Guardar en un archivo Excel
    with pd.ExcelWriter('peajes.xlsx') as writer:
        df_peajes.to_excel(writer, sheet_name='peajes')

# Uso del código
capitales = "Aguascalientes, Mexicali, La Paz, San Francisco de Campeche, Tuxtla Gutiérrez, Chihuahua, Ciudad de México, Saltillo, Colima, Victoria de Durango, Toluca, Guanajuato, Chilpancingo, Pachuca, Guadalajara, Morelia, Cuernavaca, Tepic, Monterrey, Oaxaca de Juárez, Puebla de Zaragoza, Santiago de Querétaro, Chetumal, San Luis Potosí, Culiacán, Hermosillo, Villahermosa, Ciudad Victoria, Tlaxcala de Xicohténcatl, Xalapa, Mérida, Zacatecas"
capitales = capitales.split(", ")

# Obtener matriz de peajes
peajes = obtener_peajes_por_ciudades(capitales)

# Guardar datos en un archivo Excel
guardar_peajes(capitales, peajes)

Aguascalientes a Mexicali: $1.087


KeyboardInterrupt: 